<a href="https://colab.research.google.com/github/ipeirotis/dealing_with_data/blob/master/04-Web_Scraping/B-Crawling_altmetric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import requests # This command allows us to fetch URLs
from lxml import html
import pandas as pd

url = "http://www.altmetric.com/top100/2016"
response = requests.get(url) # get the html of that url
doc = html.fromstring(response.text) # parse it and create a document

articles = doc.xpath("//li[@class='article']")

def parseArticle_xpath(article):

    title = article.xpath(".//div[@class='title']/h2/a")[0].text_content()

    url = article.xpath(".//div[@class='title']/h2/a/@href")[0]

    abstract = article.xpath(".//div[@class='abstract']")[0].text_content()

    details = article.xpath(".//a[@class='more']/@href")[0]

    nodes = article.xpath(".//div[@class='badge']//span[contains(text(), 'news stor')]/strong")
    news = nodes[0].text_content() if len(nodes)>0 else None

    nodes = article.xpath(".//div[@class='badge']//span[contains(text(), 'tweet')]/strong")
    tweets = nodes[0].text_content() if len(nodes)>0 else None

    nodes = article.xpath(".//div[@class='badge']//span[contains(text(), 'blog post')]/strong")
    blogs = nodes[0].text_content() if len(nodes)>0 else None

    nodes = article.xpath(".//div[@class='badge']//span[contains(text(), 'article on F1000')]/strong")
    f1000 = nodes[0].text_content() if len(nodes)>0 else None

    # blogs = article.xpath(".//div[@class='mentions']//span/strong")[1].text_content()
    return {
        "title": title,
        "url": url,
        "abstract": abstract,
        "details": details,
        "news_mentions": news,
        "tweets": tweets,
        "blogs": blogs,
        "f1000": f1000
    }


data = [parseArticle_xpath(article) for article in articles]
df = pd.DataFrame(data)
df



,title,url,abstract,details,news_mentions,tweets,blogs,f1000
0,United States Health Care Reform: Progress to ...,https://doi.org/10.1001/jama.2016.9797,In the first academic paper to be published by...,https://www.altmetric.com/details/9612849,315,"8,943",45,1
1,Medical errorâthe third leading cause of dea...,https://doi.org/10.1136/bmj.i2139,Medical error is not included on death certifi...,https://www.altmetric.com/details/7133536,619,"2,615",42,None
2,Observation of Gravitational Waves from a Bina...,https://doi.org/10.1103/physrevlett.116.061102,The decades-long search for gravitational wave...,https://www.altmetric.com/details/5333883,125,"4,584",48,None
3,Evidence for a Distant Giant Planet in the Sol...,https://doi.org/10.3847/0004-6256/151/2/22,Astrophysicist Mike Brown might have been part...,https://www.altmetric.com/details/5017617,536,"1,134",63,None
4,Sugar Industry and Coronary Heart Disease Rese...,https://doi.org/10.1001/jamainternmed.2016.5394,Resesarchers from UCSF report that the sugar i...,https://www.altmetric.com/details/11962797,417,"2,252",37,1
...,...,...,...,...,...,...,...,...
95,Acne and telomere length. A new spectrum betwe...,https://doi.org/10.1016/j.jid.2016.09.014,This study found a genetic link between acne a...,https://www.altmetric.com/details/12264712,221,3,2,None
96,Online social integration is associated with r...,https://doi.org/10.1073/pnas.1605554113,How many 'friends' do you have online? This st...,https://www.altmetric.com/details/13203693,201,404,8,None
97,Screening for Colorectal Cancer: US Preventive...,https://doi.org/10.1001/jama.2016.5989,New recommendations state that screening for c...,https://www.altmetric.com/details/8766061,238,522,13,None
98,Impact of meat and Lower Palaeolithic food pro...,http://rdcu.be/m3aC,The introduction of meat and new ways of prepa...,https://www.altmetric.com/details/6147547,186,279,18,None


In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

url = "http://www.altmetric.com/top100/2016"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

articles = soup.find_all('li', class_='article')

def parseArticle(article):
    # In CSS selectors, a period (.) before a name is shorthand for selecting
    # elements with that class. So .title is equivalent to class='title'
    # and .abstract is equivalent to class='abstract'.

    title = article.select_one('.title h2 a').get_text()
    url = article.select_one('.title h2 a')['href']
    abstract = article.select_one('.abstract').get_text()
    details = article.select_one('a.more')['href']

    def get_badge_text(keyword):
        node = article.select_one(f".badge span:-soup-contains('{keyword}') strong")
        return node.get_text() if node else None

    news = get_badge_text('news stor')
    tweets = get_badge_text('tweet')
    blogs = get_badge_text('blog post')
    f1000 = get_badge_text('article on F1000')

    return {
        "title": title,
        "url": url,
        "abstract": abstract,
        "details": details,
        "news_mentions": news,
        "tweets": tweets,
        "blogs": blogs,
        "f1000": f1000
    }

data = [parseArticle(article) for article in articles]
df = pd.DataFrame(data)
df



,title,url,abstract,details,news_mentions,tweets,blogs,f1000
0,United States Health Care Reform: Progress to ...,https://doi.org/10.1001/jama.2016.9797,In the first academic paper to be published by...,https://www.altmetric.com/details/9612849,315,"8,943",45,1
1,Medical errorâthe third leading cause of dea...,https://doi.org/10.1136/bmj.i2139,Medical error is not included on death certifi...,https://www.altmetric.com/details/7133536,619,"2,615",42,None
2,Observation of Gravitational Waves from a Bina...,https://doi.org/10.1103/physrevlett.116.061102,The decades-long search for gravitational wave...,https://www.altmetric.com/details/5333883,125,"4,584",48,None
3,Evidence for a Distant Giant Planet in the Sol...,https://doi.org/10.3847/0004-6256/151/2/22,Astrophysicist Mike Brown might have been part...,https://www.altmetric.com/details/5017617,536,"1,134",63,None
4,Sugar Industry and Coronary Heart Disease Rese...,https://doi.org/10.1001/jamainternmed.2016.5394,Resesarchers from UCSF report that the sugar i...,https://www.altmetric.com/details/11962797,417,"2,252",37,1
...,...,...,...,...,...,...,...,...
95,Acne and telomere length. A new spectrum betwe...,https://doi.org/10.1016/j.jid.2016.09.014,This study found a genetic link between acne a...,https://www.altmetric.com/details/12264712,221,3,2,None
96,Online social integration is associated with r...,https://doi.org/10.1073/pnas.1605554113,How many 'friends' do you have online? This st...,https://www.altmetric.com/details/13203693,201,404,8,None
97,Screening for Colorectal Cancer: US Preventive...,https://doi.org/10.1001/jama.2016.5989,New recommendations state that screening for c...,https://www.altmetric.com/details/8766061,238,522,13,None
98,Impact of meat and Lower Palaeolithic food pro...,http://rdcu.be/m3aC,The introduction of meat and new ways of prepa...,https://www.altmetric.com/details/6147547,186,279,18,None
